# 2. Citybenches: Proof of Concept 

In [14]:
#Necessary packages
import osmnx as ox
import networkx as nx
import geopandas 
import matplotlib.pyplot as plt
import pandas as pd

## Results of Data Wrangling

In [15]:
#Import data frame, adding 'New York City' to every field in Address-column
df = pd.read_excel('citybenchlocationsv03.xlsx')
df['Address'] = df['Address'] + ', New York City'

In [16]:
#Subset original data set
addressborough = df[['Address', 'Borough']]

## 2.1 Data Wrangling: Round 2

In [17]:
#Proof of Concept: ten city benches in Manhattan
addressmanhattan = addressborough[addressborough['Borough'] == 'Manhattan']
addressmanhattanh = addressmanhattan.head(10) #Limited to 10 entries

In [18]:
#Adding a column with Geocodes to df
addressmanhattang = []
for i in addressmanhattanh['Address']:
    addressmanhattang.append(ox.geocode(i))

addressmanhattanh.insert(2, 'Geocode', addressmanhattang, True)

In [19]:
#Choosing a graph that contains all ten addresses (suboptimal solution)
G = ox.graph_from_point((40.74619633333333, -73.983977), network_type = 'walk', distance = 5000, simplify = False)

In [20]:
#Adding a column with Nodes to df
addressmanhattann = []
for i in addressmanhattanh['Geocode']:
    addressmanhattann.append(ox.get_nearest_node(G, i))    
    
addressmanhattanh.insert(3, 'Node', addressmanhattann, True)

## 2.2 Distance Calculation 

In [21]:
#Loop for distance calculation (suboptimal)
distance = []
for i in addressmanhattanh['Node']:
    for j in addressmanhattanh['Node']:
        distance.append(nx.shortest_path_length(G, i, j, weight = 'length'))

distancewd = dict.fromkeys(distance)
print("The shortest distance [in m] between two benches:")
print(sorted(distancewd)[1])

The shortest distance [in m] between two benches:
89.288


## 2.3 Interpretation of our Results
At this point, we are able to calculate the shortest distance between two benches for a small subset of the set of all benches which were installed in NYC in 2019. There are several reasons why we have limited our investigation to ten city benches in Manhattan:
- The calculation of the shortest route from node to node demands a huge amount of computing power
- Not every address can be geocoded properly with "ox.geocode()". We asssume that this is due the non-standard syntax of some addresses

Although we are able to calculate the shortest distance, we are not yet able to print the respective addresses. So there is still work to do. 